In [3]:
from Bio import SeqIO
from io import StringIO
from gzip import open as gzopen
from Bio.SeqIO.FastaIO import SimpleFastaParser
import numpy as np
#import matplotlib.pyplot as plt
import os 
import pandas as pd

pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [9]:
amp_DB_fasta = "/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/amp_clustered100.22DBs.fasta"
uniref50 = "/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/uniref50.len2350.fasta"
LenFile = open('/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/non_AMPs.csv', 'w')

In [5]:

lengths = map(len, SeqIO.parse(amp_DB_fasta, 'fasta'))
df = pd.Series([x for x in lengths], name="length").to_frame()

In [6]:
df.length.describe()

count    21226.000000
mean        60.496938
std        114.696874
min         11.000000
25%         18.000000
50%         28.000000
75%         61.000000
max       2337.000000
Name: length, dtype: float64

In [7]:
df['length'].value_counts(bins=100)

(8.673, 34.26]        12220
(34.26, 57.52]         3384
(57.52, 80.78]         2620
(80.78, 104.04]        1208
(127.3, 150.56]         218
                      ...  
(1592.68, 1615.94]        0
(1639.2, 1662.46]         0
(1662.46, 1685.72]        0
(1708.98, 1732.24]        0
(1150.74, 1174.0]         0
Name: length, Length: 100, dtype: int64

In [8]:
df_count=df.groupby('length').length.count().reset_index(name='count').sort_values(['count'], ascending=False)
#df_count.to_csv('df_count.csv', index=False)
_tmp_df_count =df_count
_tmp_df_count

,length,count
9,20,1131
2,13,1119
1,12,912
7,18,873
4,15,855
...,...,...
392,448,1
394,455,1
396,460,1
397,461,1


In [30]:
_tmp_df_count.loc[ _tmp_df_count['length'] == 20]['count'].values +1

9    1128
Name: count, dtype: int64

In [47]:
_tmp_df_count['count'].values.sum() == 0

False

In [ ]:

with open(uniref50) as record_len2400:
      for _name, _seq in  SimpleFastaParser(record_len2400):
           seqLen = len(_seq)
           #update value
           _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
           if not _tmp_row.empty and  _tmp_row['count'].values > 0:
               #print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
               LenFile.write(_name+','+_seq+',' + str(seqLen) + '\n')
               _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
           else:
               if _tmp_df_count['count'].values.sum() == 0:
                   print("stop")
                   break

        


In [ ]:
"""LenFile = open('non_AMPs.csv', 'w')

with open("/mnt/c/works/RKI/uniref50.len2400.fasta") as record_len2400:
    for index, row in df_count.iterrows():
        _count= row['count']
        _length= row['length']
        print("_length:"+ str(_length))
        for _name, _seq in  SimpleFastaParser(record_len2400):
            seqLen = len(_seq)
            if _length == seqLen:
                #print("seqLen:"+ str(seqLen))
                LenFile.write(_name+','+_seq+',' + str(seqLen) + '\n')
                _count = _count - 1 
            if _count == 0:
                break;
        
        if _count != 0:
            print("Remaining:"+ str(_count) +" " + str(_length))"""


In [ ]:
## Split file 

def batch_iterator(iterator, batch_size):
    """Returns lists of length batch_size.

    This can be used on any iterator, for example to batch up
    SeqRecord objects from Bio.SeqIO.parse(...), or to batch
    Alignment objects from Bio.AlignIO.parse(...), or simply
    lines from a file handle.

    This is a generator function, and it returns lists of the
    entries from the supplied iterator.  Each list will have
    batch_size entries, although the final list may be shorter.
    """
    entry = True  # Make sure we loop once
    while entry:
        batch = []
        while len(batch) < batch_size:
            try:
                entry =  next(iterator)
            except StopIteration:
                entry = None
            if entry is None:
                # End of file
                break
            batch.append(entry)
        if batch:
            yield batch
            
record_iter = SeqIO.parse(gzopen("uniref50.fasta.gz", "rt"), "fasta")
for i, batch in enumerate(batch_iterator(record_iter, 200000)):
    filename = "group_%i.fasta" % (i + 1)
    with open(filename, "w") as handle:
        count = SeqIO.write(batch, handle, "fasta")
    print("Wrote %i records to %s" % (count, filename))

#  Step:2 Data labeling

In [ ]:
# setup
am